In [1]:
!pip3 install fasttext
!pip3 install pandoc

In [2]:
import logging
import json

import matplotlib.pyplot as plt
import numpy as np
from keras.layers import Dense
from keras.layers.core import Dropout
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import GRU
from keras.models import Sequential
from keras.preprocessing import sequence
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV

from fasttext_loader import FasttextLoader
from preprocessing_tools import inds_texts_labels, MultiLabelEncoder, MultilabelStratifiedKFold
%matplotlib inline

Using TensorFlow backend.


In [3]:
#проверка, что используется gpu
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 2459760501741778495
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 11308947866
locality {
  bus_id: 1
}
incarnation: 7933466937889870921
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"
]


In [4]:
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s %(levelname)s %(message)s')

In [5]:
%config InlineBackend.figure_format='retina'

In [6]:
seed = 7
np.random.seed(seed)

## Data loading

In [7]:
with open('nations.json', 'r') as fp:
    label_to_nation = json.load(fp)
label_to_nation = {int(k):v for k,v in label_to_nation.items()}
#label_to_nation[mlb.le.inverse_transform(j)]

In [8]:
w2v = FasttextLoader()
w2v.load("fasttext.bin")

In [9]:
X, _, y = inds_texts_labels("all_xeno_labeled.json", w2v)
w2v_weigths = w2v.weights()
mlb = MultiLabelEncoder()
y = mlb.fit_transform(y)

Num of data < min_len: 0
Num of multilabel data: 457
Num of not_xeno or unknown: 368
Num of duplicates: 0


In [10]:
max_length = max([len(elem) for elem in X])
X = sequence.pad_sequences(X, maxlen=max_length)
print("X.shape {}".format(X.shape))

X.shape (3113, 446)


In [11]:
target_shape = y.shape[1]
target_shape

7

In [12]:
#для ускорения
X = X[:141]
y = y[:141]

## Prerequisites

In [13]:
#initial
rnn_params = {"dropout": 0.5, "optimizer": "Adagrad", "epochs": 21, "batch_size": 8}

In [14]:
def rnn_model(dropout=rnn_params["dropout"], optimizer=rnn_params["optimizer"]):
    print('building network')
    global w2v, max_length, target_shape, w2v_weigths
    model = Sequential()
    model.add(Embedding(w2v.vocabulary_size() + 1, w2v.vector_dim(), input_length=max_length,
                        weights=[w2v_weigths], trainable=False))
    model.add(GRU(w2v.vector_dim()))
    model.add(Dropout(dropout))
    model.add(Dense(target_shape, activation="sigmoid"))
    model.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=["accuracy"])
    return model

In [15]:
def f1_scorer(estimator, X, y):
    preds = estimator.predict_proba(X)
    preds[preds >= 0.5] = 1
    preds[preds < 0.5] = 0
    return f1_score(y, preds, average="samples")

In [16]:
def print_grid_result(grid_result):
    print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
    for mean, stdev, param in zip(grid_result.cv_results_['mean_test_score'],
                                  grid_result.cv_results_['std_test_score'],
                                  grid_result.cv_results_['params']):
        print("%f (%f) with: %r" % (mean, stdev, param))

In [17]:
cv = MultilabelStratifiedKFold(targets=y, n_folds=3)

## Tune Batch Size and Number of Epochs

In [18]:
%%time
model = KerasClassifier(build_fn=rnn_model, verbose=True)
batch_size = [8, 16, 32]
epochs = [7, 14, 21]
param_grid = dict(batch_size=batch_size, epochs=epochs)
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=cv, scoring=f1_scorer, verbose=True)
grid_result = grid.fit(X, y)

Fold distributions are
[[  9.   1.  10.   9.  30.  15.   6.]
 [  6.   1.   6.   6.  20.  11.   4.]
 [  6.   1.   7.   6.  21.  10.   4.]]
Fitting 3 folds for each of 9 candidates, totalling 27 fits
building network
Epoch 1/7
82/82 [==============================] - 6s 69ms/step - loss: 0.5758 - acc: 0.7491
Epoch 2/7
82/82 [==============================] - 5s 66ms/step - loss: 0.4384 - acc: 0.8118
Epoch 3/7
82/82 [==============================] - 5s 66ms/step - loss: 0.3942 - acc: 0.8397
Epoch 4/7
82/82 [==============================] - 5s 66ms/step - loss: 0.3794 - acc: 0.8310
Epoch 5/7
82/82 [==============================] - 6s 67ms/step - loss: 0.3649 - acc: 0.8415
Epoch 6/7
82/82 [==============================] - 6s 71ms/step - loss: 0.3237 - acc: 0.8624
Epoch 7/7
 8/82 [=>............................] - ETA: 1s

/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)


82/82 [==============================] - 2s 20ms/step
building network
Epoch 1/7
101/101 [==============================] - 7s 67ms/step - loss: 0.5529 - acc: 0.7525
Epoch 2/7
101/101 [==============================] - 7s 65ms/step - loss: 0.4284 - acc: 0.8359
Epoch 3/7
101/101 [==============================] - 6s 64ms/step - loss: 0.3905 - acc: 0.8515
Epoch 4/7
101/101 [==============================] - 6s 60ms/step - loss: 0.3616 - acc: 0.8487
Epoch 5/7
101/101 [==============================] - 6s 60ms/step - loss: 0.3240 - acc: 0.8614
Epoch 6/7
101/101 [==============================] - 6s 60ms/step - loss: 0.2780 - acc: 0.8826
Epoch 7/7
101/101 [==============================] - 2s 19ms/step
building network
Epoch 1/7
99/99 [==============================] - 7s 66ms/step - loss: 0.5653 - acc: 0.7302
Epoch 2/7
99/99 [==============================] - 6s 65ms/step - loss: 0.4201 - acc: 0.8182
Epoch 3/7
99/99 [==============================] - 6s 65ms/step - loss: 0.3896 - acc: 0.82

101/101 [==============================] - 3s 34ms/step - loss: 0.4521 - acc: 0.8289
Epoch 3/14
101/101 [==============================] - 4s 35ms/step - loss: 0.4193 - acc: 0.8190
Epoch 4/14
101/101 [==============================] - 4s 35ms/step - loss: 0.3933 - acc: 0.8317
Epoch 5/14
101/101 [==============================] - 4s 36ms/step - loss: 0.3733 - acc: 0.8458
Epoch 6/14
101/101 [==============================] - 3s 34ms/step - loss: 0.3457 - acc: 0.8515
Epoch 7/14
101/101 [==============================] - 4s 35ms/step - loss: 0.3328 - acc: 0.8642
Epoch 8/14
101/101 [==============================] - 4s 38ms/step - loss: 0.3030 - acc: 0.8769
Epoch 9/14
101/101 [==============================] - 4s 35ms/step - loss: 0.2828 - acc: 0.8798
Epoch 10/14
101/101 [==============================] - 4s 38ms/step - loss: 0.2431 - acc: 0.9038
Epoch 11/14
101/101 [==============================] - 4s 38ms/step - loss: 0.2483 - acc: 0.9095
Epoch 12/14
101/101 [============================

82/82 [==============================] - 1s 17ms/step - loss: 0.4231 - acc: 0.8397
Epoch 6/21
82/82 [==============================] - 1s 18ms/step - loss: 0.3906 - acc: 0.8275
Epoch 7/21
82/82 [==============================] - 1s 17ms/step - loss: 0.3869 - acc: 0.8484
Epoch 8/21
82/82 [==============================] - 1s 17ms/step - loss: 0.3690 - acc: 0.8537
Epoch 9/21
82/82 [==============================] - 1s 18ms/step - loss: 0.3620 - acc: 0.8589
Epoch 10/21
82/82 [==============================] - 1s 18ms/step - loss: 0.3554 - acc: 0.8571
Epoch 11/21
82/82 [==============================] - 1s 18ms/step - loss: 0.3396 - acc: 0.8659
Epoch 12/21
82/82 [==============================] - 1s 18ms/step - loss: 0.3118 - acc: 0.8780
Epoch 13/21
82/82 [==============================] - 1s 18ms/step - loss: 0.3072 - acc: 0.8728
Epoch 14/21
82/82 [==============================] - 2s 19ms/step - loss: 0.2891 - acc: 0.8850
Epoch 15/21
82/82 [==============================] - 2s 19ms/step 

[Parallel(n_jobs=1)]: Done  27 out of  27 | elapsed: 26.8min finished


Epoch 1/21
141/141 [==============================] - 11s 76ms/step - loss: 0.5285 - acc: 0.7751
Epoch 2/21
141/141 [==============================] - 9s 66ms/step - loss: 0.3968 - acc: 0.8379
Epoch 3/21
141/141 [==============================] - 10s 68ms/step - loss: 0.3723 - acc: 0.8511
Epoch 4/21
141/141 [==============================] - 9s 65ms/step - loss: 0.3342 - acc: 0.8592
Epoch 5/21
141/141 [==============================] - 9s 65ms/step - loss: 0.2963 - acc: 0.8886
Epoch 6/21
141/141 [==============================] - 9s 66ms/step - loss: 0.2713 - acc: 0.8956
Epoch 7/21
141/141 [==============================] - 9s 66ms/step - loss: 0.2277 - acc: 0.9108
Epoch 8/21
141/141 [==============================] - 9s 65ms/step - loss: 0.1938 - acc: 0.9260
Epoch 9/21
141/141 [==============================] - 9s 66ms/step - loss: 0.1799 - acc: 0.9189
Epoch 10/21
141/141 [==============================] - 9s 65ms/step - loss: 0.1412 - acc: 0.9554
Epoch 11/21
141/141 [================

In [ ]:
print_grid_result(grid_result)
rnn_params["batch_size"] = grid_result.best_params_["batch_size"]
rnn_params["epochs"] = grid_result.best_params_["epochs"]

Best: 0.545626 using {'batch_size': 8, 'epochs': 21}
0.294326 (0.113880) with: {'batch_size': 8, 'epochs': 7}
0.471395 (0.056804) with: {'batch_size': 8, 'epochs': 14}
0.545626 (0.059523) with: {'batch_size': 8, 'epochs': 21}
0.364539 (0.027154) with: {'batch_size': 16, 'epochs': 7}
0.378014 (0.061358) with: {'batch_size': 16, 'epochs': 14}
0.480615 (0.037349) with: {'batch_size': 16, 'epochs': 21}
0.313239 (0.059365) with: {'batch_size': 32, 'epochs': 7}
0.367612 (0.033978) with: {'batch_size': 32, 'epochs': 14}
0.404255 (0.046070) with: {'batch_size': 32, 'epochs': 21}


## Tune the Training Optimization Algorithm

In [18]:
%%time
model = KerasClassifier(build_fn=rnn_model, epochs=rnn_params["epochs"], batch_size=rnn_params["batch_size"], verbose=True)
optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
param_grid = dict(optimizer=optimizer)
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=cv, scoring=f1_scorer, verbose=True)
grid_result = grid.fit(X, y)

Fold distributions are
[[  9.   1.  10.   9.  30.  15.   6.]
 [  6.   1.   6.   6.  20.  11.   4.]
 [  6.   1.   7.   6.  21.  10.   4.]]
Fitting 3 folds for each of 7 candidates, totalling 21 fits
building network
Epoch 1/21
82/82 [==============================] - 6s 71ms/step - loss: 0.6885 - acc: 0.5296
Epoch 2/21
82/82 [==============================] - 6s 68ms/step - loss: 0.6775 - acc: 0.5679
Epoch 3/21
82/82 [==============================] - 6s 68ms/step - loss: 0.6626 - acc: 0.6167
Epoch 4/21
82/82 [==============================] - 6s 69ms/step - loss: 0.6492 - acc: 0.6777
Epoch 5/21
82/82 [==============================] - 6s 69ms/step - loss: 0.6335 - acc: 0.7230
Epoch 6/21
82/82 [==============================] - 6s 67ms/step - loss: 0.6203 - acc: 0.7456
Epoch 7/21
82/82 [==============================] - 6s 67ms/step - loss: 0.6118 - acc: 0.7613
Epoch 8/21
82/82 [==============================] - 6s 68ms/step - loss: 0.6035 - acc: 0.7596
Epoch 9/21
82/82 [===============

/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)


82/82 [==============================] - 2s 20ms/step
building network
Epoch 1/21
101/101 [==============================] - 7s 67ms/step - loss: 0.6826 - acc: 0.5545
Epoch 2/21
101/101 [==============================] - 6s 63ms/step - loss: 0.6658 - acc: 0.6011
Epoch 3/21
101/101 [==============================] - 7s 65ms/step - loss: 0.6470 - acc: 0.6874
Epoch 4/21
101/101 [==============================] - 7s 64ms/step - loss: 0.6340 - acc: 0.7143
Epoch 5/21
101/101 [==============================] - 7s 66ms/step - loss: 0.6174 - acc: 0.7313
Epoch 6/21
101/101 [==============================] - 7s 66ms/step - loss: 0.6042 - acc: 0.7539
Epoch 7/21
101/101 [==============================] - 7s 65ms/step - loss: 0.5875 - acc: 0.7907
Epoch 8/21
101/101 [==============================] - 6s 64ms/step - loss: 0.5731 - acc: 0.7864
Epoch 9/21
101/101 [==============================] - 7s 65ms/step - loss: 0.5694 - acc: 0.7808
Epoch 10/21
101/101 [==============================] - 7s 67ms/st

101/101 [==============================] - 7s 65ms/step - loss: 0.0507 - acc: 0.9958
Epoch 20/21
101/101 [==============================] - 6s 64ms/step - loss: 0.0488 - acc: 0.9915
Epoch 21/21
101/101 [==============================] - 2s 19ms/step
building network
Epoch 1/21
99/99 [==============================] - 7s 71ms/step - loss: 0.5037 - acc: 0.7763
Epoch 2/21
99/99 [==============================] - 7s 71ms/step - loss: 0.4047 - acc: 0.8326
Epoch 3/21
99/99 [==============================] - 6s 65ms/step - loss: 0.3859 - acc: 0.8384
Epoch 4/21
99/99 [==============================] - 6s 65ms/step - loss: 0.3567 - acc: 0.8413
Epoch 5/21
99/99 [==============================] - 6s 65ms/step - loss: 0.3227 - acc: 0.8499
Epoch 6/21
99/99 [==============================] - 6s 65ms/step - loss: 0.2969 - acc: 0.8745
Epoch 7/21
99/99 [==============================] - 7s 66ms/step - loss: 0.2518 - acc: 0.8990
Epoch 8/21
99/99 [==============================] - 7s 66ms/step - loss: 0.

82/82 [==============================] - 6s 71ms/step - loss: 0.1420 - acc: 0.9547
Epoch 13/21
82/82 [==============================] - 6s 71ms/step - loss: 0.1192 - acc: 0.9686
Epoch 14/21
82/82 [==============================] - 6s 72ms/step - loss: 0.1068 - acc: 0.9721
Epoch 15/21
82/82 [==============================] - 6s 69ms/step - loss: 0.0865 - acc: 0.9791
Epoch 16/21
82/82 [==============================] - 6s 70ms/step - loss: 0.0689 - acc: 0.9913
Epoch 17/21
82/82 [==============================] - 5s 66ms/step - loss: 0.0633 - acc: 0.9878
Epoch 18/21
82/82 [==============================] - 5s 65ms/step - loss: 0.0558 - acc: 0.9913
Epoch 19/21
82/82 [==============================] - 6s 69ms/step - loss: 0.0582 - acc: 0.9861
Epoch 20/21
82/82 [==============================] - 6s 70ms/step - loss: 0.0494 - acc: 0.9930
Epoch 21/21
82/82 [==============================] - 2s 20ms/step
building network
Epoch 1/21
101/101 [==============================] - 8s 77ms/step - loss:

99/99 [==============================] - 7s 67ms/step - loss: 0.2799 - acc: 0.8846
Epoch 5/21
99/99 [==============================] - 7s 70ms/step - loss: 0.2491 - acc: 0.8990
Epoch 6/21
99/99 [==============================] - 7s 66ms/step - loss: 0.1855 - acc: 0.9278
Epoch 7/21
99/99 [==============================] - 6s 65ms/step - loss: 0.1431 - acc: 0.9481
Epoch 8/21
99/99 [==============================] - 7s 67ms/step - loss: 0.1206 - acc: 0.9654
Epoch 9/21
99/99 [==============================] - 7s 68ms/step - loss: 0.0813 - acc: 0.9784
Epoch 10/21
99/99 [==============================] - 7s 69ms/step - loss: 0.0574 - acc: 0.9870
Epoch 11/21
99/99 [==============================] - 7s 67ms/step - loss: 0.0410 - acc: 0.9971
Epoch 12/21
99/99 [==============================] - 7s 67ms/step - loss: 0.0313 - acc: 0.9986
Epoch 13/21
99/99 [==============================] - 7s 67ms/step - loss: 0.0463 - acc: 0.9913
Epoch 14/21
99/99 [==============================] - 6s 65ms/step -

[Parallel(n_jobs=1)]: Done  21 out of  21 | elapsed: 48.1min finished


Epoch 1/21
141/141 [==============================] - 11s 78ms/step - loss: 0.4501 - acc: 0.8055
Epoch 2/21
141/141 [==============================] - 9s 67ms/step - loss: 0.3471 - acc: 0.8531
Epoch 3/21
141/141 [==============================] - 9s 65ms/step - loss: 0.2840 - acc: 0.8713
Epoch 4/21
141/141 [==============================] - 9s 62ms/step - loss: 0.2178 - acc: 0.9129
Epoch 5/21
141/141 [==============================] - 9s 64ms/step - loss: 0.1971 - acc: 0.9250
Epoch 6/21
141/141 [==============================] - 9s 66ms/step - loss: 0.1666 - acc: 0.9422
Epoch 7/21
141/141 [==============================] - 9s 66ms/step - loss: 0.1394 - acc: 0.9483
Epoch 8/21
141/141 [==============================] - 9s 65ms/step - loss: 0.1183 - acc: 0.9676
Epoch 9/21
141/141 [==============================] - 9s 64ms/step - loss: 0.0991 - acc: 0.9696
Epoch 10/21
141/141 [==============================] - 9s 65ms/step - loss: 0.0908 - acc: 0.9777
Epoch 11/21
141/141 [=================

In [19]:
print_grid_result(grid_result)
rnn_params["optimizer"] = grid_result.best_params_["optimizer"]

Best: 0.594799 using {'optimizer': 'Adagrad'}
0.282979 (0.091983) with: {'optimizer': 'SGD'}
0.567849 (0.100834) with: {'optimizer': 'RMSprop'}
0.594799 (0.037064) with: {'optimizer': 'Adagrad'}
0.351064 (0.060982) with: {'optimizer': 'Adadelta'}
0.550591 (0.066368) with: {'optimizer': 'Adam'}
0.493617 (0.026547) with: {'optimizer': 'Adamax'}
0.550355 (0.031866) with: {'optimizer': 'Nadam'}


## Tune Dropout Regularization

In [18]:
%%time
model = KerasClassifier(build_fn=rnn_model, epochs=rnn_params["epochs"], batch_size=rnn_params["batch_size"], verbose=True)
dropout = [0.3, 0.4, 0.5, 0.6, 0.7]
param_grid = dict(dropout=dropout)
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=cv, scoring=f1_scorer, verbose=True)
grid_result = grid.fit(X, y)

Fold distributions are
[[  9.   1.  10.   9.  30.  15.   6.]
 [  6.   1.   6.   6.  20.  11.   4.]
 [  6.   1.   7.   6.  21.  10.   4.]]
Fitting 3 folds for each of 5 candidates, totalling 15 fits
building network
Epoch 1/21
82/82 [==============================] - 6s 74ms/step - loss: 0.4929 - acc: 0.7666
Epoch 2/21
82/82 [==============================] - 6s 70ms/step - loss: 0.3647 - acc: 0.8449
Epoch 3/21
82/82 [==============================] - 6s 70ms/step - loss: 0.2811 - acc: 0.8902
Epoch 4/21
82/82 [==============================] - 6s 71ms/step - loss: 0.2135 - acc: 0.9146
Epoch 5/21
82/82 [==============================] - 6s 71ms/step - loss: 0.2056 - acc: 0.9268
Epoch 6/21
82/82 [==============================] - 6s 73ms/step - loss: 0.1526 - acc: 0.9564
Epoch 7/21
82/82 [==============================] - 6s 72ms/step - loss: 0.1110 - acc: 0.9686
Epoch 8/21
82/82 [==============================] - 6s 69ms/step - loss: 0.0998 - acc: 0.9774
Epoch 9/21
82/82 [===============

/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)


82/82 [==============================] - 2s 20ms/step
building network
Epoch 1/21
101/101 [==============================] - 7s 70ms/step - loss: 0.4740 - acc: 0.7935
Epoch 2/21
101/101 [==============================] - 7s 67ms/step - loss: 0.3567 - acc: 0.8515
Epoch 3/21
101/101 [==============================] - 7s 65ms/step - loss: 0.2959 - acc: 0.8826
Epoch 4/21
101/101 [==============================] - 7s 67ms/step - loss: 0.2437 - acc: 0.9066
Epoch 5/21
101/101 [==============================] - 7s 68ms/step - loss: 0.1929 - acc: 0.9321
Epoch 6/21
101/101 [==============================] - 7s 68ms/step - loss: 0.1462 - acc: 0.9519
Epoch 7/21
101/101 [==============================] - 7s 67ms/step - loss: 0.1245 - acc: 0.9646
Epoch 8/21
101/101 [==============================] - 7s 67ms/step - loss: 0.1012 - acc: 0.9760
Epoch 9/21
101/101 [==============================] - 7s 68ms/step - loss: 0.0812 - acc: 0.9830
Epoch 10/21
101/101 [==============================] - 7s 68ms/st

101/101 [==============================] - 7s 66ms/step - loss: 0.0189 - acc: 1.0000
Epoch 20/21
101/101 [==============================] - 7s 66ms/step - loss: 0.0157 - acc: 1.0000
Epoch 21/21
101/101 [==============================] - 2s 19ms/step
building network
Epoch 1/21
99/99 [==============================] - 7s 70ms/step - loss: 0.4714 - acc: 0.7951
Epoch 2/21
99/99 [==============================] - 7s 66ms/step - loss: 0.3513 - acc: 0.8543
Epoch 3/21
99/99 [==============================] - 7s 68ms/step - loss: 0.3018 - acc: 0.8860
Epoch 4/21
99/99 [==============================] - 7s 69ms/step - loss: 0.2536 - acc: 0.8874
Epoch 5/21
99/99 [==============================] - 7s 71ms/step - loss: 0.1988 - acc: 0.9264
Epoch 6/21
99/99 [==============================] - 7s 72ms/step - loss: 0.1643 - acc: 0.9351
Epoch 7/21
99/99 [==============================] - 7s 69ms/step - loss: 0.1368 - acc: 0.9538
Epoch 8/21
99/99 [==============================] - 7s 70ms/step - loss: 0.

82/82 [==============================] - 6s 70ms/step - loss: 0.1026 - acc: 0.9756
Epoch 13/21
82/82 [==============================] - 6s 70ms/step - loss: 0.0995 - acc: 0.9739
Epoch 14/21
82/82 [==============================] - 6s 71ms/step - loss: 0.1018 - acc: 0.9686
Epoch 15/21
82/82 [==============================] - 6s 72ms/step - loss: 0.0784 - acc: 0.9861
Epoch 16/21
82/82 [==============================] - 6s 68ms/step - loss: 0.0956 - acc: 0.9739
Epoch 17/21
82/82 [==============================] - 6s 69ms/step - loss: 0.0728 - acc: 0.9808
Epoch 18/21
82/82 [==============================] - 6s 71ms/step - loss: 0.0613 - acc: 0.9878
Epoch 19/21
82/82 [==============================] - 6s 71ms/step - loss: 0.0523 - acc: 0.9948
Epoch 20/21
82/82 [==============================] - 6s 72ms/step - loss: 0.0476 - acc: 0.9861
Epoch 21/21
82/82 [==============================] - 2s 20ms/step
building network
Epoch 1/21
101/101 [==============================] - 8s 75ms/step - loss:

[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed: 35.4min finished


Epoch 1/21
141/141 [==============================] - 11s 75ms/step - loss: 0.4587 - acc: 0.7964
Epoch 2/21
141/141 [==============================] - 10s 69ms/step - loss: 0.3421 - acc: 0.8501
Epoch 3/21
141/141 [==============================] - 10s 68ms/step - loss: 0.2885 - acc: 0.8774
Epoch 4/21
141/141 [==============================] - 10s 69ms/step - loss: 0.2344 - acc: 0.9088
Epoch 5/21
141/141 [==============================] - 10s 72ms/step - loss: 0.1913 - acc: 0.9321
Epoch 6/21
141/141 [==============================] - 10s 69ms/step - loss: 0.1582 - acc: 0.9473
Epoch 7/21
141/141 [==============================] - 10s 69ms/step - loss: 0.1444 - acc: 0.9544
Epoch 8/21
141/141 [==============================] - 10s 69ms/step - loss: 0.1099 - acc: 0.9747
Epoch 9/21
141/141 [==============================] - 10s 68ms/step - loss: 0.1056 - acc: 0.9676
Epoch 10/21
141/141 [==============================] - 9s 66ms/step - loss: 0.0862 - acc: 0.9797
Epoch 11/21
141/141 [=========

In [19]:
print_grid_result(grid_result)
rnn_params["dropout"] = grid_result.best_params_["dropout"]

Best: 0.594799 using {'dropout': 0.5}
0.578014 (0.031979) with: {'dropout': 0.3}
0.555556 (0.051146) with: {'dropout': 0.4}
0.594799 (0.037064) with: {'dropout': 0.5}
0.587707 (0.051963) with: {'dropout': 0.6}
0.549409 (0.032328) with: {'dropout': 0.7}


## Result

In [20]:
rnn_params

{'batch_size': 8, 'dropout': 0.5, 'epochs': 21, 'optimizer': 'Adagrad'}